In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

categories = ['sci.space', 'comp.graphics', 'rec.sport.hockey']
dataset = fetch_20newsgroups(subset="train", remove=("headers","footers","quotes"), categories=categories)
print(dataset.data, dataset.target)

vectorizer = TfidfVectorizer(
    stop_words="english",
    max_df=0.8,
    min_df=5
)

tfidf_docs = vectorizer.fit_transform(dataset.data)

['\nMike Vernon is now 3 wins 11 losses plus that All-Star game debacle in\nafternoon games during his career...with another afternoon game with\nLos Angeles next Sunday...has the ABC deal doomed the Flames?', "I was wondering if anyone knows where I can get more information about\nthe graphics in the WingCommander series, and the RealSpace system they use.\nI think it's really awesome, and wouldn't mind being able to use similar\nfeatures in programs.  Thanks in advance.\n", 'I have 19 (2 MB worth!) uuencode\'d GIF images contain charts outlining\none of the many alternative Space Station designs being considered in\nCrystal City.  Mr. Mark Holderman works down the hall from me, and can\nbe reached for comment at (713) 483-1317, or via e-mail at\nmholderm@jscprofs.nasa.gov.\n\nMark proposed this design, which he calls "Geode" ("rough on the\noutside, but a gem on the inside") or the "ET Strongback with\nintegrated hab modules and centrifuge."  As you can see from file\ngeodeA.gif, it 

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def encode_query(query: str):
    return vectorizer.transform([query])

def search(query, top_k=5):
    q_vec = encode_query(query)                  # (1, V)
    sims = cosine_similarity(q_vec, tfidf_docs) # (1, N)
    sims = sims[0]                              
    top_idx = np.argsort(-sims)[:top_k]         # اندیس سندهای با شباهت بیشتر

    results = []
    for i in top_idx:
        results.append({
            "index": int(i),
            "similarity": float(sims[i]),
            "text": dataset.data[i][:400] + "..."      # فقط چند خط اول
        })
    return results

for r in search("space shuttle mission to repair hubble telescope", top_k=3):
    print(r["similarity"])
    print("------")
    print(r["text"])
    print("===============")



0.41859518402833606
------
Ed Campion
Headquarters, Washington, D.C.                             April 23, 1993
(Phone:  202/358-1780)

Kyle Herring
Johnson Space Center, Houston
(Phone:  713/483-5111)

RELEASE:  93-76

HUBBLE TELESCOPE SERVICING MISSION SCHEDULED FOR ELEVEN DAYS

	The December flight of Endeavour on Space Shuttle mission STS-61 to
service the Hubble Space Telescope (HST) has been scheduled as an 11 day
mis...
0.2770574735155884
------
Archive-name: space/acronyms
Edition: 8

Acronym List for sci.astro, sci.space, and sci.space.shuttle:
Edition 8, 1992 Dec 7
Last posted: 1992 Aug 27

This list is offered as a reference for translating commonly appearing
acronyms in the space-related newsgroups.  If I forgot or botched your
favorite acronym, please let me know!  Also, if there's an acronym *not*
on this list that confuses you, dr...
0.22239531388170447
------

Better idea for use of NASA Shuttle Astronauts and Crew is have them be found
lost in space after a accident wi

In [33]:
from sklearn.naive_bayes import MultinomialNB

N = len(docs)

# 2) TF-IDF روی همه‌ی داکیومنت‌ها
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_df=0.8,
    min_df=5
)
tfidf_docs = vectorizer.fit_transform(dataset.data)  # شکل: (N, V)

# 3) هر داکیومنت = یک کلاس
y_docs = np.arange(N)   # [0, 1, 2, ..., N-1]

# 4) آموزش Naive Bayes روی داکیومنت‌ها
nb_clf = MultinomialNB()
nb_clf.fit(tfidf_docs, y_docs)


# 5) تابع چاپ نتایج
def pretty_print_results_nb(query, results):
    print(f"\nQUERY: {query}")
    print("=" * 80)
    for r in results:
        print(f"Rank {r['rank']} | P(doc | query) ≈ {r['prob']:.6f} | Doc idx: {r['doc_idx']}")
        print("-" * 40)
        print(r["text"])
        print("=" * 80)


# 6) سرچ با Naive Bayes (بدون cosine)
def search_nb(query, top_k=5):
    # بردار TF-IDF کوئری
    q_vec = vectorizer.transform([query])        # (1, V)

    # احتمال هر "کلاس" = هر داکیومنت
    probs = nb_clf.predict_proba(q_vec)[0]       # شکل: (N,)

    # انتخاب top_k داکیومنت با بیشترین احتمال
    top_idx = np.argsort(-probs)[:top_k]

    results = []
    for rank, i in enumerate(top_idx, start=1):
        results.append({
            "rank": rank,
            "prob": float(probs[i]),
            "doc_idx": int(i),
            "text": dataset.data[i][:400] + "..."
        })
    return results


if __name__ == "__main__":
    query = "space shuttle mission to repair hubble telescope"
    results = search_nb(query, top_k=3)
    pretty_print_results_nb(query, results)



QUERY: space shuttle mission to repair hubble telescope
Rank 1 | P(doc | query) ≈ 0.000809 | Doc idx: 1118
----------------------------------------
Ed Campion
Headquarters, Washington, D.C.                             April 23, 1993
(Phone:  202/358-1780)

Kyle Herring
Johnson Space Center, Houston
(Phone:  713/483-5111)

RELEASE:  93-76

HUBBLE TELESCOPE SERVICING MISSION SCHEDULED FOR ELEVEN DAYS

	The December flight of Endeavour on Space Shuttle mission STS-61 to
service the Hubble Space Telescope (HST) has been scheduled as an 11 day
mis...
Rank 2 | P(doc | query) ≈ 0.000713 | Doc idx: 606
----------------------------------------
Archive-name: space/acronyms
Edition: 8

Acronym List for sci.astro, sci.space, and sci.space.shuttle:
Edition 8, 1992 Dec 7
Last posted: 1992 Aug 27

This list is offered as a reference for translating commonly appearing
acronyms in the space-related newsgroups.  If I forgot or botched your
favorite acronym, please let me know!  Also, if there's an acro